In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
BASE_PATH = 'assets'
grp = pd.read_csv(f'{BASE_PATH}/food_group_mapping.csv').set_index('code').to_dict()['grup']
features_map = pd.read_csv(f'{BASE_PATH}/features_mapping.csv').set_index('Unnamed: 0').to_dict()['name']
hindi_name = pd.read_csv(f'{BASE_PATH}/hindi_name.csv').set_index('code').to_dict()['local_name']

# 1.  PROXIMATE PRINCIPLES AND DIETARY FIBRE
## All values are expressed per 100g edible portion; All blank space in the table represent below detectable limit
def load_data(file_name):
    df = pd.read_csv(f'{BASE_PATH}/{file_name}.csv',index_col='code')
    df.columns = df.columns.map(features_map)
    df['group'] = df.index.map(grp)
    df['lang_hi'] = df.index.map(hindi_name)
    cat_cols = df.columns.difference(['Food Name','No. of Regions','group','lang_hi'])
    for col in cat_cols:
        
        df[col] = df[col].fillna("0").apply(lambda x: x.split("±")[0]).astype('float64')
    df.fillna('',inplace=True)
    with open(f'{BASE_PATH}/{file_name}.txt') as file:
        print(''.join(file.readlines()[9:]))  # Remove COPYRIGHT lines from the print 
    
    return df
        
# def calculate_uncertainty(mean, std = 0):
#     mean, std = float(mean), float(std)
#     lower_bound = mean - std
#     upper_bound = mean + std
#     return f"{lower_bound:.2f}", f"{upper_bound:.2f}"

# ## We can store the uncertainty into two seprate columns
# # df1['moisture[g]'].str.split("±").apply(lambda x: calculate_uncertainty(*x))

# def format_datatypes(cat_cols,df):
#     ## let's go ahead with the mean value
#     fill_values = {'object': '', 'float64': 0}
#     # Replace NaN values based on data type
#     for col in df.columns:
#         fill_value = fill_values.get(str('float64' if col not in cat_cols else "object"), 'object')
#         df[col] = df[col].fillna(fill_value)
#     return df

# def get_mean_value(df):
#     return df.fillna(0.0).applymap(str).applymap(lambda x: x.split("±")[0])


In [3]:
file_name = '01'

df = load_data(file_name)

# PROXIMATE PRINCIPLES AND DIETARY FIBRE
# (All values are expressed per 100g edible portion; All blank space in the table represent below detectable limit)
# - Food Code
# - Food Name
# - No. of Regions
# - Moisture [g]
# - Protein [g]
# - Ash [g]
# - Total Fat [g]
# + Dietary Fibre
#   - Total [g]
#   - Insoluble [g]
#   - Soluble [g]
# - Carbohydrate [g]
# - Energy [kJ]



In [6]:
df.head()

,Food Name,No. of Regions,Moisture,Protein,Ash,Total Fat,Dietary Fiber,Insoluble Dietary Fiber,Soluble Dietary Fiber,Carbohydrate,Energy,group,lang_hi
code,,,,,,,,,,,,,
A001,"Amaranth seed, black (Amaranthus cruentus)",1,9.89,14.59,2.78,5.74,7.02,5.76,1.26,59.98,1490.0,Cereals and Millets,Ramdana
A002,"Amaranth seed, pale brown (Amaranthus cruentus)",6,9.20,13.27,3.05,5.56,7.47,5.80,1.67,61.46,1489.0,Cereals and Millets,Ramdana
A003,Bajra (Pennisetum typhoideum),6,8.97,10.96,1.37,5.43,11.49,9.14,2.34,61.78,1456.0,Cereals and Millets,Bajra
A004,Barley (Hordeum vulgare),6,9.77,10.94,1.06,1.30,15.64,9.98,5.66,61.29,1321.0,Cereals and Millets,Jau
A005,Jowar (Sorghum vulgare),6,9.01,9.97,1.39,1.73,10.22,8.49,1.73,67.68,1398.0,Cereals and Millets,Jowar


[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [49]:
df.columns

Index(['Food Name', 'No. of Regions', 'Moisture', 'Protein', 'Ash',
       'Total Fat', 'Dietary Fiber', 'Insoluble Dietary Fiber',
       'Soluble Dietary Fiber', 'Carbohydrate', 'Energy', 'group', 'lang_hi'],
      dtype='object')

In [48]:
df[['Total Fat','Protein','Carbohydrate']].apply(lambda x: any([val/sum(x) > cutoff for cutoff,val in zip(x,[0.7,0.5,0.1])]),axis=1).sum()

233

In [27]:
# sns.boxplot(['Total Fat','Protein','Carbohydrate'],data=df)
import plotly.express as px
viz = [False for i in df.group.unique()]

plot = px.box(df,color="group", y=['Total Fat','Protein','Carbohydrate'],points="all",hover_data=['Food Name','lang_hi'],)
# Add dropdown
plot.update_layout(
    updatemenus=[
        dict(
            active=1,
            buttons=list([dict(label=grp,
                     method="update",
                     args=[{"visible": [True if pos==i else False for pos,_ in enumerate(viz)]},
                           {"title": "Data 1",
                            }]) for i,grp in enumerate(df.group.unique())
               
            ]),
        )
    ])
  
plot.show()

In [325]:

import numpy as np
from scipy.optimize import minimize

# Define the objective function
def objective_function(weights, f1, f2, p1, p2, c1, c2):
    weighted_sum = np.array([
        weights[0] * f1 + weights[1] * f2,
        weights[0] * p1 + weights[1] * p2,
        weights[0] * c1 + weights[1] * c2
    ])
    weighted_ratio = weighted_sum / weighted_sum[0]
    return -np.prod(weighted_ratio)

# Define the constraint
def constraint(weights):
    return weights[0] + weights[1] - 1


def calculate(f1,p1,c1,f2,p2,c2):
    # Define the initial guess for the weights
    weights_guess = [0.5, 0.5]


    # Define the bounds for the weights (0 <= w_i <= 1)
    bounds = [(0, 1)] * len(weights_guess)

    # Define the optimization problem
    problem = {
        'fun': objective_function,
        'x0': weights_guess,
        'args': (f1, f2, p1, p2, c1, c2),
        'bounds': bounds,
        'constraints': {'type': 'eq', 'fun': constraint}
    }

    # Solve the optimization problem
    result = minimize(**problem)

    # Print the optimal weights and the ratio of the weighted sum
    weights = result.x
    weighted_sum = np.array([
        weights[0] * f1 + weights[1] * f2,
        weights[0] * p1 + weights[1] * p2,
        weights[0] * c1 + weights[1] * c2
    ])
    weighted_ratio = weighted_sum / weighted_sum[0]
    print(f"Optimal weights: {weights[0]:.5f},  {weights[1]:.5f}")
    print(f"Ratio of the weighted sum: {tuple(zip(['Total Fat','Protein','Carbohydrate'],weighted_ratio))}")
    
# Define the variables
f1, f2 = 1, 2
p1, p2 = 3, 4
c1, c2 = 5, 6
calculate(f1,p1,c1,f2,p2,c2)

# calculate(59.98, 14.59, 5.74, 61.46, 13.27, 5.56)

Optimal weights: 1.00000,  0.00000
Ratio of the weighted sum: (('Total Fat', 1.0), ('Protein', 2.9999999999999982), ('Carbohydrate', 4.9999999999999964))


In [315]:
x = df[['Total Fat','Protein','Carbohydrate']] /100

In [316]:
list(x.iloc[10]) + list(x.iloc[19])

[0.011399999999999999, 0.07440000000000001, 0.7675, 0.0147, 0.1059, 0.6472]

In [317]:
print([1.00000000e+00 ,3.05311332e-16])

[1.0, 3.05311332e-16]


In [319]:
print(f"{1.00000000e+00:.5f}")

1.00000


In [330]:
df[df.lang_hi == "Makka"]

,Food Name,No. of Regions,Moisture,Protein,Ash,Total Fat,Dietary Fiber,Insoluble Dietary Fiber,Soluble Dietary Fiber,Carbohydrate,Energy,group,lang_hi
code,,,,,,,,,,,,,
A006,"Maize, dry (Zea mays)",6,9.26,8.80,1.17,3.77,12.24,11.29,0.94,64.77,1398.0,Cereals and Millets,Makka
A007,"Maize, tender, local (Zea mays)",6,68.29,3.57,0.38,1.40,3.67,3.23,0.43,22.69,502.0,Cereals and Millets,Makka
A008,"Maize, tender, sweet (Zea mays)",4,74.40,4.16,0.36,1.35,3.30,2.71,0.59,16.42,405.0,Cereals and Millets,Makka


In [332]:
import itertools
import pandas as pd

# Example DataFrame
dq = pd.DataFrame({'a1': [1, 2, 3], 'a2': [4, 5, 6],
                   'b1': [7, 8, 9], 'b2': [10, 11, 12],
                   'c1': [13, 14, 15], 'c2': [16, 17, 18]})

# Define the weights
weights = [0.4, 0.6]

# Calculate the maximum ratio of the weighted sums for each pair of rows
results = []
for pair in itertools.combinations(dq.iterrows(), 2):
    row1 = pair[0][1]
    row2 = pair[1][1]
    weighted_sum1 = weights[0] * row1['a1'] + weights[1] * row1['a2']
    weighted_sum2 = weights[0] * row2['a1'] + weights[1] * row2['a2']
    ratio1 = weighted_sum1 / weighted_sum2
    weighted_sum1 = weights[0] * row1['b1'] + weights[1] * row1['b2']
    weighted_sum2 = weights[0] * row2['b1'] + weights[1] * row2['b2']
    ratio2 = weighted_sum1 / weighted_sum2
    weighted_sum1 = weights[0] * row1['c1'] + weights[1] * row1['c2']
    weighted_sum2 = weights[0] * row2['c1'] + weights[1] * row2['c2']
    ratio3 = weighted_sum1 / weighted_sum2
    results.append((ratio1, ratio2, ratio3))

# Create a DataFrame with the results
results_dq = pd.DataFrame(results, columns=['ratio1', 'ratio2', 'ratio3'])
print(results_dq)

     ratio1    ratio2    ratio3
0  0.736842  0.897959  0.936709
1  0.583333  0.814815  0.880952
2  0.791667  0.907407  0.940476


In [333]:
dq

,a1,a2,b1,b2,c1,c2
0,1,4,7,10,13,16
1,2,5,8,11,14,17
2,3,6,9,12,15,18


In [334]:
results_dq

,ratio1,ratio2,ratio3
0,0.736842,0.897959,0.936709
1,0.583333,0.814815,0.880952
2,0.791667,0.907407,0.940476


In [62]:
import pandas as pd
import dash
from dash import dcc
from dash import html
import plotly.express as px



# Create sample data
data = {'Food Name': ['Apple', 'Banana', 'Orange', 'Steak', 'Salmon', 'Tofu'],
        'Total Fat': [0.2, 0.3, 0.2, 12.9, 6.5, 4.8],
        'Protein': [0.3, 1.1, 0.9, 26.1, 22.2, 8.2],
        'Carbohydrate': [13.8, 22.8, 12.3, 0.0, 0.0, 2.0],
        'group': ['Fruits', 'Fruits', 'Fruits', 'Meat', 'Meat', 'Vegetarian']}
df = pd.DataFrame(data)

# Create Dash app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div([
    dcc.Dropdown(
        id='group-dropdown',
        options=[{'label': group, 'value': group} for group in df['group'].unique()],
        value=df['group'].unique()[0]
    ),
    html.Br(),
    dcc.Graph(id='box-plot'),
    html.Br(),
    html.Div(id='table-container')
])

# Define callbacks
@app.callback(
    dash.dependencies.Output('box-plot', 'figure'),
    [dash.dependencies.Input('group-dropdown', 'value')]
)
def update_box_plot(selected_group):
    filtered_df = df[df['group'] == selected_group]
    fig = px.box(filtered_df, y=['Total Fat', 'Protein', 'Carbohydrate'])
    return fig

@app.callback(
    dash.dependencies.Output('table-container', 'children'),
    [dash.dependencies.Input('group-dropdown', 'value')]
)
def update_table(selected_group):
    filtered_df = df[df['group'] == selected_group]
    sorted_df = filtered_df.sort_values(['Total Fat', 'Protein', 'Carbohydrate', 'Food Name'], ascending=[True, False, False, True])
    table = html.Table([
        html.Thead(html.Tr([html.Th(col) for col in sorted_df.columns])),
        html.Tbody([html.Tr([html.Td(sorted_df.iloc[i][col]) for col in sorted_df.columns]) for i in range(len(sorted_df))])
    ])
    return table

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True,port=8093)


Dash is running on http://127.0.0.1:8093/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Traceback (most recent call last):
  File "/Users/pranavjha/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/pranavjha/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 18, in <module>
    from IPython.core.application import (
  File "/Users/pranavjha/opt/anaconda3/lib/python3.9/site-packages/IPython/__init__.py", line 56, in <module>
    from .terminal.embed import embed
  File "/Users/pranavjha/opt/anaconda3/lib/python3.9/site-packages/IPython/terminal/embed.py", line 16, in <module>
    from IPython.terminal.interactiveshell import TerminalInteractiveShell
  File "/Users/pranavjha/opt/anaconda3/lib/python3.9/site-packages/IPython/terminal/interactiveshell.py", line 35, in <module>
    from .debugger import TerminalPdb, Pdb
  File "/Users/pranavjha/opt/anaconda3/lib/python3.9/site-packages/IPython/terminal/debugger.py", line 6, in <module>
    from IPython.core.compl

SystemExit: 1

In [3]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df.country.unique(), 'Canada', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.country==value]
    return px.line(dff, x='year', y='pop')

if __name__ == '__main__':
    app.run_server(debug=True,port=8077)

    

Dash is running on http://127.0.0.1:8077/

 * Serving Flask app '__main__'
 * Debug mode: on


Traceback (most recent call last):
  File "/Users/pranavjha/Library/CloudStorage/OneDrive-Personal/swastik/nutrion/.venv/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/pranavjha/Library/CloudStorage/OneDrive-Personal/swastik/nutrion/.venv/lib/python3.9/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/Users/pranavjha/Library/CloudStorage/OneDrive-Personal/swastik/nutrion/.venv/lib/python3.9/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/Users/pranavjha/Library/CloudStorage/OneDrive-Personal/swastik/nutrion/.venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 678, in initialize
    self.init_sockets()
  File "/Users/pranavjha/Library/CloudStorage/OneDrive-Personal/swastik/nutrion/.venv/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 317, in init_sockets
    self.shell

SystemExit: 1

/Users/pranavjha/Library/CloudStorage/OneDrive-Personal/swastik/nutrion/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
pip install jupyter-dash

Note: you may need to restart the kernel to use updated packages.
